# BLOCO 1: IMPORTAÇÃO DE BIBLIOTECAS

In [10]:
# OS: Biblioteca para interação com sistema operacional (manipulação de arquivos e diretórios)
import os

In [11]:
# OpenCV: Biblioteca para processamento de imagens e visão computacional
import cv2

In [12]:
# NumPy: Biblioteca para computação científica com arrays multidimensionais
import numpy as np

In [13]:
# Pandas: Biblioteca para manipulação e análise de dados (dataframes)
import pandas as pd

In [14]:
# Scikit-learn: Biblioteca para machine learning em Python
from sklearn.model_selection import StratifiedKFold # Validação cruzada estratificada
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Métricas de avaliação
from sklearn.neighbors import KNeighborsClassifier # Algoritmo KNN
from sklearn.ensemble import RandomForestClassifier # Algoritmo Random Forest
from sklearn.neural_network import MLPClassifier # Rede Neural MLP

In [15]:
# Joblib: Biblioteca para salvar e carregar modelos treinados
from joblib import dump

# BLOCO 2: CARREGAMENTO E PRÉ-PROCESSAMENTO DOS DADOS

In [16]:
# Define o diretório onde estão armazenadas as imagens do dataset
data_dir = r"imagens\indie\pi\dataset\v20220930"

# Tamanho para redimensionamento das imagens (16x16 pixels)
img_size = 16

# Listas para armazenar features (X) e labels (y)
X, y = [], []

# Processo de carregamento das imagens:
    # 1. Percorre todos os subdiretórios do dataset
    # 2. Identifica a vogal baseada no nome da pasta
    # 3. Carrega e processa cada imagem
    # 4. Converte imagens em vetores de features

for root, dirs, files in os.walk(data_dir):
    # Extrai o nome da pasta pai para identificar a vogal
    pasta_vogal = os.path.basename(os.path.dirname(root))
    letra = pasta_vogal[0].lower() if pasta_vogal else None
    
    # Processa pastas de vogais (a, e, i, o, u)
    if letra in ['a','e','i','o','u']:
        for file in files:
            # Filtra apenas arquivos de imagem
            if file.lower().endswith(('.png','.jpg','.jpeg')):
                img_path = os.path.join(root, file)
                
                # Carrega imagem em escala de cinza
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
                # Redimensiona imagem para tamanho fixo
                img = cv2.resize(img, (img_size, img_size))
                
                # Achata a matriz 16x16 em vetor 1D de 256 elementos
                X.append(img.flatten())
                
                # Cria label binário: 1 para vogal 'i', 0 para outras vogais
                y.append(1 if letra == 'i' else 0)

# Converte listas para arrays NumPy para melhor performance
X = np.array(X)
y = np.array(y)

print("Total de imagens carregadas:", len(X))

Total de imagens carregadas: 10000


# BLOCO 3: Execução

In [17]:
# Configurações do experimento
n_splits = 3      # Número de folds na validação cruzada
random_state = 42  # Semente aleatória para reproducibilidade

# Avalia um modelo usando validação cruzada
    # Parâmetros:
    # model: modelo de machine learning a ser avaliado
    # X: array com features das imagens
    # y: array com labels
    # kf: objeto de validação cruzada
# Retorna:
    # Tupla com médias das métricas (acurácia, precision, recall, f1)

def evaluate_model(model, X, y, kf):

    # Listas para armazenar métricas de cada fold
    accs, precs, recs, f1s = [], [], [], []
    
    # Executa validação cruzada fold por fold
    for train_idx, test_idx in kf.split(X, y):
        # Treina o modelo com dados de treino do fold atual
        model.fit(X[train_idx], y[train_idx])
        
        # Faz predições com dados de teste do fold atual
        preds = model.predict(X[test_idx])
        
        # Calcula métricas e armazena nas listas
        accs.append(accuracy_score(y[test_idx], preds))
        precs.append(precision_score(y[test_idx], preds))
        recs.append(recall_score(y[test_idx], preds))
        f1s.append(f1_score(y[test_idx], preds))
    
    # Retorna médias das métricas entre todos os folds
    return np.mean(accs), np.mean(precs), np.mean(recs), np.mean(f1s)

# StratifiedKFold: Garante que cada fold tenha a mesma proporção de classes que o dataset original, importante para problemas com classes desbalanceadas

kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

# KNN: Classificador baseado em distância entre vizinhos mais próximos
model_knn = KNeighborsClassifier(n_neighbors=3)

# MLP: Rede Neural Artificial com uma camada oculta de 100 neurônios
model_mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200)

# Random Forest: Ensemble de múltiplas árvores de decisão
model_rf  = RandomForestClassifier(n_estimators=100, random_state=42)

# Dicionário para armazenar resultados
results = {}

# Loop de avaliação para cada modelo:
    # 1. KNN - Baseado em distância
    # 2. MLP - Rede Neural
    # 3. Random Forest - Ensemble de árvores

for name, model in [("KNN", model_knn), ("MLP", model_mlp), ("RandomForest", model_rf)]:
    print(f"Treinando e avaliando {name}...")
    
    # Avalia o modelo usando validação cruzada
    acc, prec, rec, f1 = evaluate_model(model, X, y, kf)
    
    # Armazena resultados no dicionário
    results[name] = (acc, prec, rec, f1)
    
    # Salva o modelo treinado em arquivo para uso futuro
    dump(model, f"{name}_final_model.pkl")

# Cria DataFrame do Pandas para visualização organizada dos resultados
df = pd.DataFrame(results, index=["Acurácia", "Precision", "Recall", "F1"]).T
print(df)

# Salvamento dos resultados:
    # - NPZ: Formato NumPy para carregamento rápido em Python
    # - CSV: Formato tabular legível para humanos e outras ferramentas

# Salva em formato NumPy
np.savez("avaliacao_modelos.npz", **results)

# Salva em formato CSV
df.to_csv("avaliacao_modelos.csv")

print("Resultados da avaliação salvos em avaliacao_modelos.npz e avaliacao_modelos.csv")

Treinando e avaliando KNN...
Treinando e avaliando MLP...
Treinando e avaliando RandomForest...
              Acurácia  Precision    Recall        F1
KNN           0.969600   0.894557  0.961500  0.926787
MLP           0.918498   0.853490  0.761320  0.762729
RandomForest  0.983400   0.974200  0.942001  0.957801
Resultados da avaliação salvos em avaliacao_modelos.npz e avaliacao_modelos.csv


# Bloco 4: Validação da avaliação corretamente salva

In [18]:
# Carrega resultados salvos
data = np.load("avaliacao_modelos.npz")
for model in data:
    acc, prec, rec, f1 = data[model]
    print(model, acc, prec, rec, f1)


KNN 0.9695997494170533 0.8945573861863337 0.9615004809907358 0.9267869728533845
MLP 0.918497698600094 0.8534902708382979 0.7613197905551728 0.7627288228290116
RandomForest 0.9834000196660337 0.9742001750196065 0.9420012216114165 0.9578008737240035
